In [11]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNWithMeans

# 1. Define the dataset (as provided)
ratings_dict = { 
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1], 
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'], 
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3]
} 
df = pd.DataFrame(ratings_dict)

# 2. Define the reader and load the dataset using Surprise
reader = Reader(rating_scale=(1, 5)) 
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

# 3. Setup item-based collaborative filtering using KNN
sim_options = {     
    "name": "cosine",         # Cosine similarity
    "user_based": False,      # Item-based filtering
} 
algo = KNNWithMeans(sim_options=sim_options)

# 4. Build the training set and train the model
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

# 5. Predict ratings for items that the user hasn't rated yet
def recommend_movies(user_id, num_recommendations=5):
    # Get the list of items (movies) the user has not rated
    rated_items = [item for (item, _) in trainingSet.ur[trainingSet.to_inner_uid(user_id)]]
    
    # Generate predictions for all items the user has not rated
    predictions = []
    for item in trainingSet.all_items():
        if item not in rated_items:
            prediction = algo.predict(user_id, trainingSet.to_raw_iid(item))
            predictions.append((item, prediction.est))
    
    # Sort predictions by estimated rating in descending order
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)
    
    # Return the top N recommendations
    return recommendations[:num_recommendations]

# 6. Recommend movies for user 'E' (or any other user)
user_id = 'E'
recommendations = recommend_movies(user_id, 5)

# 7. Display the top 5 recommendations
print(f"Top 5 movie recommendations for user {user_id}:")
for item, rating in recommendations:
    print(f"Movie ID: {item}, Predicted Rating: {rating:.2f}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Top 5 movie recommendations for user E:
Movie ID: 1, Predicted Rating: 4.15
